In [ ]:
import heapq
graph_raw=[]
with open(r'Data\g1.txt','r') as f:
    graph_raw=f.readlines()
edge_list_label = [(x.split()[0],x.split()[1],int(x.split()[2])) for x in graph_raw[1:] ]
vertex_label={ v:c for c, v in enumerate(set([x[0] for x in edge_list_label]).union(set([x[1] for x in edge_list_label])), 1)}
vertex_count=int(graph_raw[0].split()[0])
edge_list=[(vertex_label[x[0]],vertex_label[x[1]],x[2]) for x in edge_list_label]

In [ ]:
#Johnson's ALgo
#Step1, add pseudo vertex p, and p to all other vertex has a direct edge with length 0
for v in range(vertex_count):
    edge_list.append((0,v+1,0))
#Step1.1, create a hashtable edge_list_destdict for edge_list, with key as dest vertex, value as source vertex and edge length
edge_list_destdict={}
for edge in edge_list:
    if edge[1] not in edge_list_destdict:
        edge_list_destdict[edge[1]]={edge[0]:edge[2]}
    else:
        edge_list_destdict[edge[1]][edge[0]]=edge[2] 
#Step2, Run Bellman-Ford Algo to get the shortest path of all vertex to pseudo vertex
#bf_distance_array is a 2D array, cell[i,j] stores the distance of vertex j to source vertex(pseudo vertex 0) with max i edges
bf_distance_array=[]
for i in range(vertex_count+2):
    #initialize the bf_distance_array, for 0 edge budget, except for 0 vertex to 0 vertex has a distance of 0, all other vertex has a distance infinity (empty) to vertex 0
    bf_distance_array.append([])
    if i==0:
        bf_distance_array[0].append(0)
        for j in range(vertex_count):
            bf_distance_array[0].append('')
    else:
        for j in range(vertex_count+1):
            #for any vertex j, the shortest distance to source vertex 0, at edge budget i, is the minimum of below:
            #shortest distance of vertex j to source vertex 0, at edge budget i-1
            #shortest distance of vertex u (which has a direct length to j) to source vertex 0, at edge budget i-1 + edge length from u to j
            if j==0:
                #For pseudo source vertex 0, since no other vertex has a path to it, the distance to itself is always 0
                bf_distance_array[i].append(0)
            else:
                tmp_min=bf_distance_array[i-1][j]
                for source in edge_list_destdict[j]:
                    if (tmp_min != '') & (bf_distance_array[i-1][source] != ''):
                        tmp_min = min(tmp_min, bf_distance_array[i-1][source]+edge_list_destdict[j][source])
                    elif bf_distance_array[i-1][source] != '':
                        tmp_min = bf_distance_array[i-1][source]+edge_list_destdict[j][source]
                bf_distance_array[i].append(tmp_min)
#Step2.1, check if there are any negative cycle
negative_cycle_flag=False
for j in range(vertex_count+1):
    if bf_distance_array[vertex_count][j] != bf_distance_array[vertex_count+1][j]:
        negative_cycle_flag=True
        break
if negative_cycle_flag==True:
    print ('Negative Cycle Detected')
#for the case when no negative cycle detected, calculate all pair shortest path, else return
else:
    #Step3, reweight the edges, create a hashtable edge_list_sourcedict for edge_list, with key as source vertex, value as dest vertex and reweighted edge length
    #edge_list_sourcedict should not include the pseudo vertex 0
    edge_list_sourcedict={}
    for edge in edge_list:
        if edge[0]!=0:
            if edge[0] not in edge_list_sourcedict:
                edge_list_sourcedict[edge[0]]={edge[1]:edge[2]+bf_distance_array[vertex_count][edge[0]]-bf_distance_array[vertex_count][edge[1]]}
            else:
                edge_list_sourcedict[edge[0]][edge[1]]=edge[2]+bf_distance_array[vertex_count][edge[0]]-bf_distance_array[vertex_count][edge[1]]
    #Step4, run Dijkstra Algo
    #all_pair_shortest_path_dict is a dict of dimension n*(n-1)
    all_pair_shortest_path_dict={}
    for i in range(vertex_count+1):
        if ((i!=0) & (i in edge_list_sourcedict)):
            #There is no vertex with lable 0, empty iteration of i=0
            #For a sink vertex, i.e. i not in edge_list_sourcedict, it cannot reach any other vertex
            #i's iteration, calculates the shortest distance for all vertex to vertex i
            all_pair_shortest_path_dict[i]={}
            #Step4.1, dijkstra_distance_heap stores for all the vertext whose shortest distance to source vertex has not calculated
            #initialize dijkstra_distance_heap with all vertext that has a direct edge to source vertex i
            dijkstra_distance_heap=[]
            #dijkstra_distance_dict maps the dijkstra distance value to vertex index
            #initialize dijkstra_distance_dict with all vertext that has a direct edge to source vertex i
            dijkstra_distance_dict={}
            for vertex in edge_list_sourcedict[i]:
                heapq.heappush(dijkstra_distance_heap, (edge_list_sourcedict[i][vertex],vertex))
                dijkstra_distance_dict[vertex]=edge_list_sourcedict[i][vertex]
            while len(dijkstra_distance_heap) != 0:
                #Step4.2, pop from dijkstra_distance_heap, populate the poped node to all_pair_shortest_path_array
                v=heapq.heappop(dijkstra_distance_heap)
                while ((v[1] in all_pair_shortest_path_dict[i]) & (len(dijkstra_distance_heap) != 0)):
                    v=heapq.heappop(dijkstra_distance_heap)
                if ((v[1] not in all_pair_shortest_path_dict[i]) & (v[1] != i)):
                    all_pair_shortest_path_dict[i][v[1]]=v[0]
                    #Step4.3 for all vertex that has a direct edge from the poped vertex, update the vertex's value in dijkstra_distance_heap
                    if v[1] in edge_list_sourcedict:
                    #if a connected vertex is a sink vertex, the dijkstra_distance_heap remains unchanged
                        for connected_v in edge_list_sourcedict[v[1]]:
                            #if the connected vertex already has a dijkstra distance calculated
                            #refresh the dijkstra distance with popped vertex's dijkstra distance + popped vertex's distance to the connected vertex if necessary
                            if connected_v in dijkstra_distance_dict:
                                if dijkstra_distance_dict[connected_v]>dijkstra_distance_dict[v[1]]+edge_list_sourcedict[v[1]][connected_v]:
                                    dijkstra_distance_dict[connected_v]=dijkstra_distance_dict[v[1]]+edge_list_sourcedict[v[1]][connected_v]
                                    heapq.heappush(dijkstra_distance_heap, (dijkstra_distance_dict[connected_v],connected_v))
                            else:
                                dijkstra_distance_dict[connected_v]=dijkstra_distance_dict[v[1]]+edge_list_sourcedict[v[1]][connected_v]
                                heapq.heappush(dijkstra_distance_heap, (dijkstra_distance_dict[connected_v],connected_v))
    #derive the true all pair shortest path by reverting the vertex reweight
    #min_shortest_path keep track of the minimum of shortest path among all pair shortest path
    min_shortest_path=''
    for s in all_pair_shortest_path_dict:
        for v in all_pair_shortest_path_dict[s]:
            all_pair_shortest_path_dict[s][v]=all_pair_shortest_path_dict[s][v]+bf_distance_array[vertex_count][v]-bf_distance_array[vertex_count][s]
            if min_shortest_path == '':
                min_shortest_path = all_pair_shortest_path_dict[s][v]
            else:
                min_shortest_path = min(min_shortest_path, all_pair_shortest_path_dict[s][v])


In [198]:
min_shortest_path

-3215

In [ ]:
#Tested, with assumption, that no multiple edge between the same vertex